In [1]:
import tensorlayer as tl
import tensorflow as tf
import rawpy

import model
import utils

from PIL import Image
import numpy as np
import glob
import importlib
from PIL import Image

## Set config

In [2]:
import config as _config
importlib.reload(_config)
from config import config, log_config

###====================== HYPER-PARAMETERS ===========================###
## Adam
batch_size = config.TRAIN.batch_size
lr_init = config.TRAIN.lr_init
beta1 = config.TRAIN.beta1
## initialize G
n_epoch_init = config.TRAIN.n_epoch_init
## adversarial learning (SRGAN)
n_epoch = config.TRAIN.n_epoch
lr_decay = config.TRAIN.lr_decay
decay_every = config.TRAIN.decay_every
training_dir = config.TRAIN.training_dir
pretrain_checkpoint = config.TRAIN.pretrain_checkpoint
train_data_dir = config.TRAIN.train_data_dir

crop_num = config.TRAIN.crop_num

tl.files.exists_or_mkdir(training_dir)
tl.files.exists_or_mkdir(pretrain_checkpoint)

[TL] [!] training_dir exists ...
[TL] [!] checkpoint exists ...


True

## Define Model

In [3]:
import model as _model
importlib.reload(_model)
import utils as _utils
importlib.reload(_utils)
from model import *
from utils import *

In [4]:
t_image = tf.placeholder('float32', [None, 96, 96, 3], name='t_image_input_to_SRGAN_generator')
t_target_image = tf.placeholder('float32', [None, 384, 384, 1], name='t_target_image')

net_g = SRGAN_g(t_image, is_train=True, reuse=False)
net_d, logits_real = SRGAN_d(t_target_image, is_train=True, reuse=False)
_, logits_fake = SRGAN_d(net_g.outputs, is_train=True, reuse=True)

# net_g.print_params(False)￼

net_g.print_params(False)
# net_g.print_layers()
net_d.print_params(False)
# net_d.print_layers()

## test inference
net_g_test = SRGAN_g(t_image, is_train=False, reuse=True)

[TL] InputLayer  SRGAN_g/in: (?, 96, 96, 3)
[TL] Conv2d SRGAN_g/n64s1/c: n_filter: 64 filter_size: (3, 3) strides: (1, 1) pad: SAME act: relu
[TL] Conv2d SRGAN_g/n64s1/c1/0: n_filter: 64 filter_size: (3, 3) strides: (1, 1) pad: SAME act: No Activation
[TL] BatchNormLayer SRGAN_g/n64s1/b1/0: decay: 0.900000 epsilon: 0.000010 act: relu is_train: True
[TL] Conv2d SRGAN_g/n64s1/c2/0: n_filter: 64 filter_size: (3, 3) strides: (1, 1) pad: SAME act: No Activation
[TL] BatchNormLayer SRGAN_g/n64s1/b2/0: decay: 0.900000 epsilon: 0.000010 act: No Activation is_train: True
[TL] ElementwiseLayer SRGAN_g/b_residual_add/0: size: (?, 96, 96, 64) fn: add
[TL] Conv2d SRGAN_g/n64s1/c1/1: n_filter: 64 filter_size: (3, 3) strides: (1, 1) pad: SAME act: No Activation
[TL] BatchNormLayer SRGAN_g/n64s1/b1/1: decay: 0.900000 epsilon: 0.000010 act: relu is_train: True
[TL] Conv2d SRGAN_g/n64s1/c2/1: n_filter: 64 filter_size: (3, 3) strides: (1, 1) pad: SAME act: No Activation
[TL] BatchNormLayer SRGAN_g/n64s1/

[TL] Conv2d SRGAN_g/n64s1/c/m: n_filter: 64 filter_size: (3, 3) strides: (1, 1) pad: SAME act: No Activation
[TL] BatchNormLayer SRGAN_g/n64s1/b/m: decay: 0.900000 epsilon: 0.000010 act: No Activation is_train: True
[TL] ElementwiseLayer SRGAN_g/add3: size: (?, 96, 96, 64) fn: add
[TL] Conv2d SRGAN_g/n256s1/1_new: n_filter: 256 filter_size: (3, 3) strides: (1, 1) pad: SAME act: No Activation
[TL] SubpixelConv2d  SRGAN_g/pixelshufflerx2/1_new: scale: 2 n_out_channel: 64 act: relu
[TL] Conv2d SRGAN_g/n256s1/2_new: n_filter: 256 filter_size: (3, 3) strides: (1, 1) pad: SAME act: No Activation
[TL] SubpixelConv2d  SRGAN_g/pixelshufflerx2/2_new: scale: 2 n_out_channel: 64 act: relu
[TL] Conv2d SRGAN_g/out_new: n_filter: 1 filter_size: (1, 1) strides: (1, 1) pad: SAME act: tanh
[TL] WARNING: Function: `tensorlayer.layers.utils.set_name_reuse` (in file: /home/hwa125/.local/lib/python3.6/site-packages/tensorlayer/layers/utils.py) is deprecated and will be removed after 2018-06-30.
Instructions

[TL]   param  25: SRGAN_g/n64s1/b1/2/moving_mean:0 (64,)              float32_ref
[TL]   param  26: SRGAN_g/n64s1/b1/2/moving_variance:0 (64,)              float32_ref
[TL]   param  27: SRGAN_g/n64s1/c2/2/kernel:0 (3, 3, 64, 64)     float32_ref
[TL]   param  28: SRGAN_g/n64s1/b2/2/beta:0 (64,)              float32_ref
[TL]   param  29: SRGAN_g/n64s1/b2/2/gamma:0 (64,)              float32_ref
[TL]   param  30: SRGAN_g/n64s1/b2/2/moving_mean:0 (64,)              float32_ref
[TL]   param  31: SRGAN_g/n64s1/b2/2/moving_variance:0 (64,)              float32_ref
[TL]   param  32: SRGAN_g/n64s1/c1/3/kernel:0 (3, 3, 64, 64)     float32_ref
[TL]   param  33: SRGAN_g/n64s1/b1/3/beta:0 (64,)              float32_ref
[TL]   param  34: SRGAN_g/n64s1/b1/3/gamma:0 (64,)              float32_ref
[TL]   param  35: SRGAN_g/n64s1/b1/3/moving_mean:0 (64,)              float32_ref
[TL]   param  36: SRGAN_g/n64s1/b1/3/moving_variance:0 (64,)              float32_ref
[TL]   param  37: SRGAN_g/n64s1/c2/3/ker

[TL]   param 129: SRGAN_g/n64s1/b2/12/gamma:0 (64,)              float32_ref
[TL]   param 130: SRGAN_g/n64s1/b2/12/moving_mean:0 (64,)              float32_ref
[TL]   param 131: SRGAN_g/n64s1/b2/12/moving_variance:0 (64,)              float32_ref
[TL]   param 132: SRGAN_g/n64s1/c1/13/kernel:0 (3, 3, 64, 64)     float32_ref
[TL]   param 133: SRGAN_g/n64s1/b1/13/beta:0 (64,)              float32_ref
[TL]   param 134: SRGAN_g/n64s1/b1/13/gamma:0 (64,)              float32_ref
[TL]   param 135: SRGAN_g/n64s1/b1/13/moving_mean:0 (64,)              float32_ref
[TL]   param 136: SRGAN_g/n64s1/b1/13/moving_variance:0 (64,)              float32_ref
[TL]   param 137: SRGAN_g/n64s1/c2/13/kernel:0 (3, 3, 64, 64)     float32_ref
[TL]   param 138: SRGAN_g/n64s1/b2/13/beta:0 (64,)              float32_ref
[TL]   param 139: SRGAN_g/n64s1/b2/13/gamma:0 (64,)              float32_ref
[TL]   param 140: SRGAN_g/n64s1/b2/13/moving_mean:0 (64,)              float32_ref
[TL]   param 141: SRGAN_g/n64s1/b2/13/

[TL] Conv2d SRGAN_g/n64s1/c1/1: n_filter: 64 filter_size: (3, 3) strides: (1, 1) pad: SAME act: No Activation
[TL] BatchNormLayer SRGAN_g/n64s1/b1/1: decay: 0.900000 epsilon: 0.000010 act: relu is_train: False
[TL] Conv2d SRGAN_g/n64s1/c2/1: n_filter: 64 filter_size: (3, 3) strides: (1, 1) pad: SAME act: No Activation
[TL] BatchNormLayer SRGAN_g/n64s1/b2/1: decay: 0.900000 epsilon: 0.000010 act: No Activation is_train: False
[TL] ElementwiseLayer SRGAN_g/b_residual_add/1: size: (?, 96, 96, 64) fn: add
[TL] Conv2d SRGAN_g/n64s1/c1/2: n_filter: 64 filter_size: (3, 3) strides: (1, 1) pad: SAME act: No Activation
[TL] BatchNormLayer SRGAN_g/n64s1/b1/2: decay: 0.900000 epsilon: 0.000010 act: relu is_train: False
[TL] Conv2d SRGAN_g/n64s1/c2/2: n_filter: 64 filter_size: (3, 3) strides: (1, 1) pad: SAME act: No Activation
[TL] BatchNormLayer SRGAN_g/n64s1/b2/2: decay: 0.900000 epsilon: 0.000010 act: No Activation is_train: False
[TL] ElementwiseLayer SRGAN_g/b_residual_add/2: size: (?, 96, 96

[TL] SubpixelConv2d  SRGAN_g/pixelshufflerx2/2_new: scale: 2 n_out_channel: 64 act: relu
[TL] Conv2d SRGAN_g/out_new: n_filter: 1 filter_size: (1, 1) strides: (1, 1) pad: SAME act: tanh


## Define training op

In [5]:
d_loss1 = tl.cost.sigmoid_cross_entropy(logits_real, tf.ones_like(logits_real), name='d1')
d_loss2 = tl.cost.sigmoid_cross_entropy(logits_fake, tf.zeros_like(logits_fake), name='d2')
d_loss = d_loss1 + d_loss2

g_gan_loss = 1e-3 * tl.cost.sigmoid_cross_entropy(logits_fake, tf.ones_like(logits_fake), name='g')
mse_loss = tl.cost.mean_squared_error(net_g.outputs, t_target_image, is_mean=True)
# vgg_loss = 2e-6 * tl.cost.mean_squared_error(vgg_predict_emb.outputs, vgg_target_emb.outputs, is_mean=True)

# g_loss = mse_loss + vgg_loss + g_gan_loss
g_loss = mse_loss + g_gan_loss

g_vars = tl.layers.get_variables_with_name('SRGAN_g', True, True)
d_vars = tl.layers.get_variables_with_name('SRGAN_d', True, True)

with tf.variable_scope('learning_rate'):
    lr_v = tf.Variable(lr_init, trainable=False)
## Pretrain
g_optim_init = tf.train.AdamOptimizer(lr_v, beta1=beta1).minimize(mse_loss, var_list=g_vars)
## SRGAN
g_optim = tf.train.AdamOptimizer(lr_v, beta1=beta1).minimize(g_loss, var_list=g_vars)
d_optim = tf.train.AdamOptimizer(lr_v, beta1=beta1).minimize(d_loss, var_list=d_vars)

[TL]   [*] geting variables with SRGAN_g
[TL]   got   0: SRGAN_g/n64s1/c/kernel:0   (3, 3, 3, 64)
[TL]   got   1: SRGAN_g/n64s1/c/bias:0   (64,)
[TL]   got   2: SRGAN_g/n64s1/c1/0/kernel:0   (3, 3, 64, 64)
[TL]   got   3: SRGAN_g/n64s1/b1/0/beta:0   (64,)
[TL]   got   4: SRGAN_g/n64s1/b1/0/gamma:0   (64,)
[TL]   got   5: SRGAN_g/n64s1/c2/0/kernel:0   (3, 3, 64, 64)
[TL]   got   6: SRGAN_g/n64s1/b2/0/beta:0   (64,)
[TL]   got   7: SRGAN_g/n64s1/b2/0/gamma:0   (64,)
[TL]   got   8: SRGAN_g/n64s1/c1/1/kernel:0   (3, 3, 64, 64)
[TL]   got   9: SRGAN_g/n64s1/b1/1/beta:0   (64,)
[TL]   got  10: SRGAN_g/n64s1/b1/1/gamma:0   (64,)
[TL]   got  11: SRGAN_g/n64s1/c2/1/kernel:0   (3, 3, 64, 64)
[TL]   got  12: SRGAN_g/n64s1/b2/1/beta:0   (64,)
[TL]   got  13: SRGAN_g/n64s1/b2/1/gamma:0   (64,)
[TL]   got  14: SRGAN_g/n64s1/c1/2/kernel:0   (3, 3, 64, 64)
[TL]   got  15: SRGAN_g/n64s1/b1/2/beta:0   (64,)
[TL]   got  16: SRGAN_g/n64s1/b1/2/gamma:0   (64,)
[TL]   got  17: SRGAN_g/n64s1/c2/2/kernel:0  

## restore model

In [6]:
sess = tf.Session(config=tf.ConfigProto(allow_soft_placement=True,
                                        log_device_placement=False))

In [7]:
tl.layers.initialize_global_variables(sess)
if tl.files.load_and_assign_npz(sess=sess, name=training_dir + '/g_srgan.npz', network=net_g) is None:
    load_pretrain_model(sess=sess, npz_file=pretrain_checkpoint + '/g_srgan.npz', network=net_g)
tl.files.load_and_assign_npz(sess=sess, name=training_dir + '/d_srgan.npz', network=net_d)

[TL] WARNING: Function: `tensorlayer.layers.utils.initialize_global_variables` (in file: /home/hwa125/.local/lib/python3.6/site-packages/tensorlayer/layers/utils.py) is deprecated and will be removed after 2018-09-30.
Instructions for updating: This API is deprecated in favor of `tf.global_variables_initializer`

[TL] ERROR: file training_dir/g_srgan.npz doesn't exist.
[TL] [*] Load checkpoint/g_srgan.npz SUCCESS!
[TL] ERROR: file training_dir/d_srgan.npz doesn't exist.


## Training
### initialize G

In [8]:
sess.run(tf.assign(lr_v, lr_init))
print(" ** fixed learning rate: %f (for init G)" % lr_init)

 ** fixed learning rate: 0.000100 (for init G)


In [9]:
train_data_list = tl.files.load_file_list(train_data_dir, regx = '.*.arw', printable = False)

In [10]:
for epoch in range(0, n_epoch_init + 1):
    epoch_time = time.time()
    total_mse_loss, n_iter = 0, 0

    for idx in range(0, len(train_data_list), batch_size):
        step_time = time.time()
        
        batch_file_name = train_data_list[idx: idx+batch_size]
        inputs_rgbs, label_raws = get_inputs_labels(train_data_dir, batch_file_name, 2)
        
        ## update G
        errM, _ = sess.run([mse_loss, g_optim_init], {t_image: inputs_rgbs, t_target_image: label_raws})
        print("Epoch [%2d/%2d] %4d time: %4.4fs, mse: %.8f " % (epoch, n_epoch_init, n_iter, time.time() - step_time, errM))
        total_mse_loss += errM
        n_iter += 1
    log = "[*] Epoch: [%2d/%2d] time: %4.4fs, mse: %.8f" % (epoch, n_epoch_init, time.time() - epoch_time, total_mse_loss / n_iter)
    print(log)

    ## quick evaluation on train set
#     if (epoch != 0) and (epoch % 10 == 0):
#         out = sess.run(net_g_test.outputs, {t_image: sample_imgs_96})  #; print('gen sub-image:', out.shape, out.min(), out.max())
#         print("[*] save images")
#         tl.vis.save_images(out, [ni, ni], save_dir_ginit + '/train_%d.png' % epoch)

    ## save model
    if (epoch != 0) and (epoch % 10 == 0):
        tl.files.save_npz(net_g.all_params, name=training_dir + '/g_srgan_init.npz', sess=sess)

Epoch [ 0/100]    0 time: 5.1405s, mse: 0.00431612 
Epoch [ 0/100]    1 time: 0.9554s, mse: 0.00110007 
Epoch [ 0/100]    2 time: 0.9247s, mse: 0.00241852 
Epoch [ 0/100]    3 time: 0.9285s, mse: 0.00706428 
[*] Epoch: [ 0/100] time: 7.9493s, mse: 0.00372475
Epoch [ 1/100]    0 time: 0.9258s, mse: 0.00024928 
Epoch [ 1/100]    1 time: 0.9266s, mse: 0.02691215 
Epoch [ 1/100]    2 time: 0.9395s, mse: 0.00140690 
Epoch [ 1/100]    3 time: 0.8895s, mse: 0.03431595 
[*] Epoch: [ 1/100] time: 3.6818s, mse: 0.01572107
Epoch [ 2/100]    0 time: 0.9324s, mse: 0.00060681 
Epoch [ 2/100]    1 time: 0.9282s, mse: 0.00479323 
Epoch [ 2/100]    2 time: 0.9430s, mse: 0.00100325 
Epoch [ 2/100]    3 time: 0.8869s, mse: 0.00275586 
[*] Epoch: [ 2/100] time: 3.6907s, mse: 0.00228978
Epoch [ 3/100]    0 time: 0.9547s, mse: 0.00097979 
Epoch [ 3/100]    1 time: 0.9330s, mse: 0.00444230 
Epoch [ 3/100]    2 time: 0.9270s, mse: 0.01230310 
Epoch [ 3/100]    3 time: 0.8923s, mse: 0.00263789 
[*] Epoch: [ 3/

[TL] [*] Saved
Epoch [31/100]    0 time: 0.9989s, mse: 0.00182154 
Epoch [31/100]    1 time: 0.9518s, mse: 0.00174835 
Epoch [31/100]    2 time: 0.9240s, mse: 0.04317007 
Epoch [31/100]    3 time: 0.8883s, mse: 0.00708913 
[*] Epoch: [31/100] time: 3.7633s, mse: 0.01345727
Epoch [32/100]    0 time: 0.9336s, mse: 0.00536372 
Epoch [32/100]    1 time: 0.9470s, mse: 0.00283436 
Epoch [32/100]    2 time: 0.9261s, mse: 0.00244184 
Epoch [32/100]    3 time: 0.9189s, mse: 0.00173305 
[*] Epoch: [32/100] time: 3.7261s, mse: 0.00309324
Epoch [33/100]    0 time: 0.9451s, mse: 0.00123014 
Epoch [33/100]    1 time: 0.9611s, mse: 0.00139862 
Epoch [33/100]    2 time: 1.0318s, mse: 0.00157274 
Epoch [33/100]    3 time: 0.9161s, mse: 0.00120164 
[*] Epoch: [33/100] time: 3.8543s, mse: 0.00135078
Epoch [34/100]    0 time: 1.0102s, mse: 0.00101302 
Epoch [34/100]    1 time: 0.9683s, mse: 0.00084487 
Epoch [34/100]    2 time: 0.9365s, mse: 0.01576240 
Epoch [34/100]    3 time: 0.9229s, mse: 0.00139163 


Epoch [62/100]    0 time: 0.9336s, mse: 0.00154106 
Epoch [62/100]    1 time: 0.9350s, mse: 0.00064188 
Epoch [62/100]    2 time: 0.9369s, mse: 0.00055710 
Epoch [62/100]    3 time: 0.8867s, mse: 0.00037875 
[*] Epoch: [62/100] time: 3.6925s, mse: 0.00077970
Epoch [63/100]    0 time: 0.9348s, mse: 0.00078128 
Epoch [63/100]    1 time: 0.9303s, mse: 0.00170283 
Epoch [63/100]    2 time: 0.9264s, mse: 0.00107444 
Epoch [63/100]    3 time: 0.8866s, mse: 0.00024128 
[*] Epoch: [63/100] time: 3.6784s, mse: 0.00094996
Epoch [64/100]    0 time: 0.9275s, mse: 0.00100131 
Epoch [64/100]    1 time: 0.9350s, mse: 0.00865388 
Epoch [64/100]    2 time: 0.9347s, mse: 0.00071146 
Epoch [64/100]    3 time: 0.8953s, mse: 0.00037998 
[*] Epoch: [64/100] time: 3.6928s, mse: 0.00268666
Epoch [65/100]    0 time: 0.9450s, mse: 0.00038980 
Epoch [65/100]    1 time: 0.9482s, mse: 0.00160033 
Epoch [65/100]    2 time: 0.9280s, mse: 0.00060800 
Epoch [65/100]    3 time: 0.8814s, mse: 0.00376350 
[*] Epoch: [65/

Epoch [93/100]    0 time: 0.9262s, mse: 0.00015552 
Epoch [93/100]    1 time: 0.9434s, mse: 0.00025694 
Epoch [93/100]    2 time: 0.9356s, mse: 0.00100932 
Epoch [93/100]    3 time: 0.8882s, mse: 0.00017151 
[*] Epoch: [93/100] time: 3.6937s, mse: 0.00039832
Epoch [94/100]    0 time: 0.9224s, mse: 0.00033664 
Epoch [94/100]    1 time: 0.9351s, mse: 0.00045634 
Epoch [94/100]    2 time: 0.9382s, mse: 0.00527926 
Epoch [94/100]    3 time: 0.8883s, mse: 0.00016641 
[*] Epoch: [94/100] time: 3.6842s, mse: 0.00155966
Epoch [95/100]    0 time: 0.9358s, mse: 0.00056490 
Epoch [95/100]    1 time: 0.9473s, mse: 0.00056219 
Epoch [95/100]    2 time: 0.9365s, mse: 0.00173014 
Epoch [95/100]    3 time: 0.8949s, mse: 0.00015542 
[*] Epoch: [95/100] time: 3.7148s, mse: 0.00075316
Epoch [96/100]    0 time: 0.9359s, mse: 0.00012418 
Epoch [96/100]    1 time: 0.9308s, mse: 0.00032499 
Epoch [96/100]    2 time: 0.9267s, mse: 0.00053001 
Epoch [96/100]    3 time: 0.8850s, mse: 0.00011032 
[*] Epoch: [96/

In [16]:
for epoch in range(0, 2 + 1):
        ## update learning rate
        if epoch != 0 and (epoch % decay_every == 0):
            new_lr_decay = lr_decay**(epoch // decay_every)
            sess.run(tf.assign(lr_v, lr_init * new_lr_decay))
            log = " ** new learning rate: %f (for GAN)" % (lr_init * new_lr_decay)
            print(log)
        elif epoch == 0:
            sess.run(tf.assign(lr_v, lr_init))
            log = " ** init lr: %f  decay_every_init: %d, lr_decay: %f (for GAN)" % (lr_init, decay_every, lr_decay)
            print(log)

        epoch_time = time.time()
        total_d_loss, total_g_loss, n_iter = 0, 0, 0

        for idx in range(0, len(train_data_list), batch_size):
            step_time = time.time()
            batch_file_name = train_data_list[idx: idx+batch_size]
            inputs_rgbs, label_raws = get_inputs_labels(train_data_dir, batch_file_name, 2)
            ## update D
            errD, _ = sess.run([d_loss, d_optim], {t_image: inputs_rgbs, t_target_image: label_raws})
            ## update G
            errG, errM, errA, _ = sess.run([g_loss, mse_loss, g_gan_loss, g_optim],
                                                 {t_image: inputs_rgbs, t_target_image: label_raws})
            print("Epoch [%2d/%2d] %4d time: %4.4fs, d_loss: %.8f g_loss: %.8f (mse: %.6f adv: %.6f)" %
                  (epoch, n_epoch, n_iter, time.time() - step_time, errD, errG, errM, errA))
            total_d_loss += errD
            total_g_loss += errG
            n_iter += 1

        log = "[*] Epoch: [%2d/%2d] time: %4.4fs, d_loss: %.8f g_loss: %.8f" % (epoch, n_epoch, time.time() - epoch_time, total_d_loss / n_iter,
                                                                                total_g_loss / n_iter)
        print(log)

        ## quick evaluation on train set
#         if (epoch != 0) and (epoch % 10 == 0):
#             out = sess.run(net_g_test.outputs, {t_image: sample_imgs_96})  #; print('gen sub-image:', out.shape, out.min(), out.max())
#             print("[*] save images")
#             tl.vis.save_images(out, [ni, ni], save_dir_gan + '/train_%d.png' % epoch)

        ## save model
        if (epoch != 0) and (epoch % 10 == 0):
            tl.files.save_npz(net_g.all_params, name=training_dir + '/g_srgan.npz'.format(tl.global_flag['mode']), sess=sess)
            tl.files.save_npz(net_d.all_params, name=training_dir + '/d_srgan.npz'.format(tl.global_flag['mode']), sess=sess)

 ** init lr: 0.000100  decay_every_init: 1000, lr_decay: 0.100000 (for GAN)
Epoch [ 0/2000]    0 time: 1.5955s, d_loss: 2.13979721 g_loss: 0.03410675 (mse: 0.000407 adv: 0.033700)
Epoch [ 0/2000]    1 time: 1.5941s, d_loss: 2.55675888 g_loss: 0.00868321 (mse: 0.000189 adv: 0.008494)
Epoch [ 0/2000]    2 time: 1.5318s, d_loss: 2.59092546 g_loss: 0.02898122 (mse: 0.006354 adv: 0.022627)
Epoch [ 0/2000]    3 time: 1.4577s, d_loss: 2.33331823 g_loss: 0.00952866 (mse: 0.000309 adv: 0.009220)
[*] Epoch: [ 0/2000] time: 6.1793s, d_loss: 2.40519994 g_loss: 0.02032496
Epoch [ 1/2000]    0 time: 1.5098s, d_loss: 1.96535134 g_loss: 0.01540101 (mse: 0.000368 adv: 0.015033)
Epoch [ 1/2000]    1 time: 1.5604s, d_loss: 0.69386184 g_loss: 0.01694207 (mse: 0.003474 adv: 0.013468)
Epoch [ 1/2000]    2 time: 1.5769s, d_loss: 1.97620749 g_loss: 0.03274266 (mse: 0.018687 adv: 0.014056)
Epoch [ 1/2000]    3 time: 1.4568s, d_loss: 7.11636019 g_loss: 0.00418333 (mse: 0.000659 adv: 0.003524)
[*] Epoch: [ 1/200